# Summarizing Fault Codes with The Data Science Package

At the end of the week, month, or quarter, you can easily integrate information about the trips that were taken during that time period. 

In [0]:
import pandas as pd

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


We can get information about the fault codes that occured during trips within a monthly period. For instance, we may want to keep track of the fault codes that appear during trips, which vehicles have the most fault codes, and when warning lights appear. 

Firstly, we can get all of the trips and join them with the fault codes table, to place all fault codes with the trips where they occured. 

In [0]:
projectname='PUT YOUR PROJECT NAME HERE'

In [0]:
TripsandFCs= """
Select T.*, Fault_Time, Code, ActiveFlag, F_Speed 
  From 
    (Select TripId, SerialNo, StartTime, StopTime, DrivingDuration, StopDuration, Distance, IdlingDuration 
       From `geotab-dsp-$name.Interpolated.Trips` 
      Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
    )T 
Left Join
    (Select SerialNo, DateTime AS Fault_Time, Code, ActiveFlag, Speed AS F_Speed 
      From `geotab-dsp-$name.Interpolated.Fault` 
     Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
  )F 
    On T.SerialNo=F.SerialNo and Fault_Time between StartTime and StopTime
"""


In [0]:
df_TaFC=pd.io.gbq.read_gbq(TripsandFCs.replace('$name', projectname), project_id='geotab-dsp-'+projectname ,dialect='standard')

In order to find out how many trips in the past week had faults, we can write a separate query that shows you how many trips had at least one fault code register during the trip

In [0]:
TripswFCs= """
With Trips As 
  (Select T.*, Fault_Time, Code, ActiveFlag, F_Speed 
    From 
    (
    Select TripId, SerialNo, StartTime, StopTime, DrivingDuration, StopDuration, Distance, IdlingDuration 
       From `geotab-dsp-$name.Interpolated.Trips` 
      Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
    )T 
    Left Join 
    (
    Select SerialNo, DateTime AS Fault_Time, Code, ActiveFlag, Speed AS F_Speed 
      From `geotab-dsp-$name.Interpolated.Fault` 
     Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
    ) F 
    On T.SerialNo=F.SerialNo and Fault_Time between StartTime and StopTime)


  Select No_Faults, Count(TripId) No_Trips 
    From
      (
      Select TripId, Count(Fault_Time) as No_Faults
        From Trips
    Group by TripId
      )
   Where No_Faults>0
Group by No_Faults
Order by No_Faults
"""

In [0]:
df_TwFC=pd.io.gbq.read_gbq(TripswFCs.replace('$name', projectname),project_id='geotab-dsp-'+projectname, dialect='standard')

We may want to look at Vehicles and Trips that had multiple fault codes, and understand what is happening there a little bit more. We can visualize a few summary charts based on a query that takes fault codes and places them within the trip they occured. We can also take the last timestamp of the trip to find out when the most recent fault code occured. 

In [0]:
TripswMFC= """
With Trips As (
  Select T.*, Fault_Time, Code AS FaultCode, ActiveFlag, F_Speed 
    From 
    (
    Select TripId, SerialNo, StartTime, StopTime, DrivingDuration, StopDuration, Distance, IdlingDuration 
      From `geotab-dsp-$name.Interpolated.Trips` 
      Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
    ) T 
  Left Join 
  (
    Select SerialNo, DateTime AS Fault_Time, Code, ActiveFlag, Speed AS F_Speed 
      From `geotab-dsp-$name.Interpolated.Fault` 
      Where _PartitionTime between Timestamp(Date_Trunc(Date_Sub(Current_Date(), Interval 1 Month), Month))
        and Timestamp(Date_Sub(Date_Trunc(Current_DAte(), Month), Interval 1 Day))
  ) F 
  On T.SerialNo=F.SerialNo and Fault_Time between StartTime and StopTime)

,ManyFaults As 
(
  Select * 
  From
  (
    Select TripId, Count(Fault_Time) as No_Faults 
      From Trips
      Group by TripId)
  Where No_Faults>9)

Select Distinct SerialNo, TripId, FaultCode, MostRecent,  No_Records, Description 
  From 
  (
  Select Distinct TripId, SerialNo, FaultCode, Max(Fault_Time) as MostRecent, Count(*) No_Records 
    From Trips 
    Where Tripid In (Select Tripid From ManyFaults)
    Group by SerialNo, FaultCode, TripId
  ) A 
  Inner Join ( 
  Select Code, Description 
    From `geotab-dsp-$name.Interpolated.FaultCodeLookup` 
    Where _PARTITIONTIME between Timestamp(Date '2018-01-01') and Timestamp(Date '2018-12-31')
  ) B 
  On FaultCode=Code
"""

In [0]:
df_TwMFC=pd.io.gbq.read_gbq(TripswMFC.replace('$name', projectname), project_id='geotab-dsp-'+projectname, dialect='standard')

 The above query provides you with a summary of all of the trips with multiple fault codes. You can visualize all of these findings in a datastudio dashboard: 
 https://datastudio.google.com/u/0/reporting/1KEbkhyuUlmDl4RkW2j9uhq-nb--aeWbH/page/vMad